In [13]:
import pandas
import numpy as np
import import_ipynb
import FeatureStore

In [14]:
from sklearn.preprocessing import StandardScaler

#https://stackoverflow.com/questions/35723472/how-to-use-sklearn-fit-transform-with-pandas-and-return-dataframe-instead-of-num
def scaleDataFrame(dataframe, debugPrint = False):
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(dataframe)
    dataframe = pandas.DataFrame(scaled_features, index=dataframe.index, columns=dataframe.columns)
    
    #check
    if debugPrint == True:
        for column in dataframe:
            print("[%s] mean = %.4f, variance = %.4f" % 
                  (column, dataframe[column].mean(), dataframe[column].var()))
    
    return dataframe, scaler.mean_, scaler.scale_

In [25]:
results = {}
for period in [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]:
    results[period] = {}
    for penalty in [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]:
        results[period][penalty] = {}
        for kernel in ['linear', 'rbf']:
            results[period][penalty][kernel] = 0.0

In [26]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score

from datetime import datetime

# C = 1000.0 algorithm hangs!
C = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]
#C = {1.0}
#grid = {'C': np.power(10.0, np.arange(-4, 0))}

sigma1 = 0
mu1 = 0
clf1 = 0
accs = {}

for period in [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]:
    X_train = FeatureStore.getTrainDataset(period)
    y_train = FeatureStore.getTrainTarget(period)
    
    X_train, sigma, mu = scaleDataFrame(X_train, debugPrint = False)
    for penalty in [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0]:
        for kernel in ['linear', 'rbf']:
            print("trying C = %f, period %d, kernel = %s" % (penalty, period, kernel))
            kf = KFold(n_splits=3, shuffle=False, random_state=241)

            #print(sigma)
            #print(mu)


            clf = svm.SVC(C = penalty, kernel = kernel, gamma='auto', random_state=241)
            if (penalty == 1.0 and period == 40):
                sigma1 = sigma
                mu1 = mu
                clf1 = clf
            #N = 800
            #XX = X_train.iloc[:N, :]
            #yy = y_train[:N]
            XX = X_train
            yy = y_train
            #print("from", datetime.now())
            scores = cross_val_score(clf, XX, yy, scoring='accuracy', cv=kf)
            #print(scores)
            print("accuracy=", scores.mean())
            #print("to", datetime.now())
            #accs[period].append(scores.mean())
            if period in accs and not (accs[period] is None):
                accs[period].append(scores.mean())
            else:
                accs[period] = [scores.mean()]
            
            results[period][penalty][kernel] = scores.mean()
        
for k in accs:
    accs[k] = np.max(accs[k])

#print(accs)

trying C = 0.000100, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 0.000100, period 1, kernel = rbf
accuracy= 0.5572259421409563
trying C = 0.001000, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 0.001000, period 1, kernel = rbf
accuracy= 0.5572259421409563
trying C = 0.010000, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 0.010000, period 1, kernel = rbf
accuracy= 0.5572259421409563
trying C = 0.100000, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 0.100000, period 1, kernel = rbf
accuracy= 0.5572259421409563
trying C = 1.000000, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 1.000000, period 1, kernel = rbf
accuracy= 0.5468012275731823
trying C = 10.000000, period 1, kernel = linear
accuracy= 0.5572259421409563
trying C = 10.000000, period 1, kernel = rbf
accuracy= 0.5354510043780581
trying C = 0.000100, period 2, kernel = linear
accuracy= 0.5941497124216671
trying C = 0.000100, period 

accuracy= 0.9347234741179501
trying C = 0.010000, period 250, kernel = linear
accuracy= 0.9347234741179501
trying C = 0.010000, period 250, kernel = rbf
accuracy= 0.9347234741179501
trying C = 0.100000, period 250, kernel = linear
accuracy= 0.9347234741179501
trying C = 0.100000, period 250, kernel = rbf
accuracy= 0.9347234741179501
trying C = 1.000000, period 250, kernel = linear
accuracy= 0.9347234741179501
trying C = 1.000000, period 250, kernel = rbf
accuracy= 0.9347234741179501
trying C = 10.000000, period 250, kernel = linear
accuracy= 0.9347234741179501
trying C = 10.000000, period 250, kernel = rbf
accuracy= 0.9347234741179501


In [27]:
for k, v in accs.items():
    print("%d;%f" % (k, v))
    

print(clf1, sigma1, mu1)

import json
with open('resultSVM.json', 'w') as fp:
    json.dump(results, fp)

1;0.557226
2;0.594150
3;0.612099
5;0.612129
10;0.626314
20;0.663203
40;0.737942
60;0.765358
120;0.839160
250;0.934723
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=241, shrinking=True, tol=0.001,
    verbose=False) [1.05107978e-03 9.80585321e-04 1.37135076e+08 1.37707874e+08
 2.03311258e+00 1.56745506e+01] [1.27345881e-02 1.29274745e-02 6.57771957e+07 6.61035530e+07
 1.40476317e+00 8.69552487e+00]


In [17]:
def backtestStrategy():
    pass